In [122]:
import json
import glob
import csv
import string

In [123]:
filepath = 'comics2char.csv'

In [124]:
out_put = open(filepath, "w+", newline='')
datafile = csv.writer(out_put)      
header = ['comic_id','char_id']
datafile.writerow(header)

18

In [125]:
files =  glob.glob('./comic_json/*.json')

In [126]:
with open(files[0]) as data_file:
    data = json.load(data_file)['data']['results']
    

In [127]:
data[0]['characters']['available'] > data[0]['characters']['returned'] 

False

In [128]:
for i in data[0]['characters']['items']:
    print(i['resourceURI'].split('/')[-1])

1010370


In [129]:
i = 0
api = []
not_api = []
for fn in files:
    with open(fn) as data_file:
        data = json.load(data_file)['data']['results']
        for comic in data:
            idd = comic['id']
            avail = comic['characters']['available']
            returned = data[0]['characters']['returned'] 
            if avail > 20:
                api.append(idd)
                continue
            
            not_api.append(idd)
            if(len(comic['characters']['items']) == 0):
                i = i + 1
                
            for char in comic['characters']['items']:
                row = []
                row.append(idd)
                row.append(char['resourceURI'].split('/')[-1])
                datafile.writerow(row)
            
                
        
        


In [130]:
api

[8555,
 13911,
 14382,
 1903,
 23514,
 10453,
 18113,
 13876,
 14345,
 21618,
 14373,
 16209,
 16222,
 14330,
 2270,
 10585,
 10584,
 10580,
 23890,
 1460,
 1311,
 3995,
 13816,
 13682,
 10492,
 1672,
 13181,
 14281,
 14379,
 23511,
 12386,
 13848,
 19571,
 10505,
 12148,
 17487,
 17457,
 5844,
 10590,
 10589,
 10588,
 10286,
 13929,
 5705,
 22058,
 21813,
 6156,
 17509,
 13920,
 8551,
 8550,
 13918,
 10498,
 17540,
 18453,
 8560,
 17501,
 17490,
 23217,
 17491,
 35509,
 15524,
 4802,
 8549,
 8559,
 1083,
 1083,
 19821,
 5132,
 5033,
 21813,
 21430,
 21295,
 4115,
 1587,
 12120,
 6156,
 2371,
 35528,
 13741,
 23512,
 1185,
 10504,
 1292,
 17454,
 5589,
 5369,
 5280,
 10494,
 12360,
 20913,
 4461,
 4412,
 4514,
 4241,
 13229,
 18172,
 42117,
 40090,
 14318,
 1780,
 1795,
 17523,
 17512,
 30015,
 20731,
 12422,
 15801,
 15800,
 15802,
 17532,
 17530,
 14313,
 14312,
 13845,
 10502,
 1079,
 1033,
 4004,
 12371,
 12370,
 13229,
 10581,
 4216,
 4100,
 4217,
 21085,
 1038,
 13836,
 14300,
 1

In [131]:
len(api)

151

In [132]:
len(not_api)

36608

In [133]:
i

18757

In [134]:
out_put.close()

In [135]:
import requests
import hashlib
import datetime
import string
import csv

In [136]:
public_key = "2990a1aae96d40df9612767f23e2cf01"
private_key = "3952fc6b62f02a30c86ee6ced917550f08f743ec"

In [137]:
def get_TS_HASH( pub_key, priv_key):
    ts = str(datetime.datetime.now()).replace(" ", "").replace(".", "").replace("-", "").replace(":", "")
    m = hashlib.md5()
    h = ts + priv_key + pub_key
    m.update( h.encode('utf-8'))
    h = m.hexdigest()
    return ts, h
    

In [138]:
def make_url(base_url,limit,offset, pub_key, priv_key):
    ts, h = get_TS_HASH( pub_key, priv_key)
    get_url = base_url + '?ts=' + ts + '&limit=' + str(limit) + '&offset=' + str(offset) + "&apikey=" + pub_key + '&hash=' + h
    return get_url

In [249]:
out_put = open(filepath, "a", newline='')
datafile = csv.writer(out_put) 

In [250]:
comic_id = api[:]

In [251]:
count = 0

In [252]:
comic_charID_url = 'http://gateway.marvel.com:80/v1/public/comics/%s/characters'

In [253]:
r = requests.get(make_url(comic_charID_url % str(comic_id[1]), 100, count, public_key, private_key))

In [254]:
data = r.json()['data']
total = data['total']

In [255]:
idd = comic_id.index(5589)

In [256]:
idd = idd + 2

In [257]:
comic_id = comic_id[idd:len(comic_id)]

In [258]:
for i in range(len(comic_id)):
    count = 0
    url = (comic_charID_url % comic_id[i])
    r = requests.get(make_url(url, 100, count, public_key, private_key))
    if(r.status_code != 200):
        print("something wrong!!! error code of %d"% r.status_code)
        break
    data = r.json()['data']
    total = data['total']
    while(1):
        count = count + data['count']
        for char in data['results']:
            row = []
            row.append(comic_id[i])
            row.append(char['id'])
            datafile.writerow(row)
        if(count > total):
            break
        if(r.status_code != 200):
            print("something wrong!!! error code of %d"% r.status_code)
            break
        data = r.json()['data']
        
        
        
        
        
    
    
    

In [259]:
out_put.close()